### Part 1. Algorithmic Implementation - Steepest Descent

In [34]:
# Install libraries
%matplotlib inline
import numpy as np
from sympy import *
from scipy import stats
from scipy.optimize import fmin
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import time

### 1) func = x1^2 + x2^2 + x3^2      and     x0 = 1,1,1

In [45]:
################################################################################################
#                                        Trivariate Calculation                                #
################################################################################################

# Start at Example:
#x = np.matrix('1;1;1')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2, x3 = symbols('x1:4')
fn = x1**2 + x2**2 + x3**2

# Lambdify objective function
f = lambdify('x1, x2, x3', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2) 
fnDiffx3 = fn.diff(x3) 
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2], [fnDiffx3]])

# Lambdify
fprime = lambdify('x1, x2, x3', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2) ; fnHessx13 = diff(diff(fn, x1),x3)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) ; fnHessx23 = diff(diff(fn, x2),x3)
fnHessx31 = diff(diff(fn, x3),x1) ; fnHessx32 = diff(diff(fn, x3),x2) ; fnHessx33 = diff(diff(fn, x3),x3) 
fnHess = Matrix([[fnHessx11, fnHessx12, fnHessx13], [fnHessx21, fnHessx22, fnHessx23], [fnHessx31, fnHessx32, fnHessx33]])

# Lambdify
fHess = lambdify('x1, x2, x3', fnHess)

# Calculate Example
#f(x.item((0,0)) , x.item((1,0)) , x.item((2,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))

In [46]:
# Start at
x = np.matrix('1;1;1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = 0.00001
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)) , x.item((2,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))

# Calculate runtime
starttime = time.time()

# Formula for gradint descent
#while np.linalg.norm(xnew - xold) > e:
while np.linalg.norm(xnew - xold) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0)) , xold.item((2,0))))
    
    # If gradient is 0 then we stop either way. Or else it will break the code.
    if(np.linalg.norm(fprimeVal) == 0) :
        break;
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[1]
 [1]
 [1]]


Iteration:  1
Direction:  [[-2]
 [-2]
 [-2]]
Step Length:  0.5
New Value:  [[ 0.]
 [ 0.]
 [ 0.]]




Local minimum occurs at: [[ 0.]
 [ 0.]
 [ 0.]]
Number of steps: 1
runtime:  0.007005929946899414


### 2) func = x1^2 + 2*x2^2 - 2*x1*x2 - 2*x2  and x0 = 0,0

In [16]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('0;0')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = x1**2 + 2*x2**2 - 2*x1*x2 - 2*x2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [23]:
# Start at
x = np.matrix('0;0')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = 0.00001
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for gradint descent
while np.linalg.norm(xnew - xold) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
    
    # If gradient is 0 then we stop either way. Or else it will break the code.
    if(np.linalg.norm(fprimeVal) == 0) :
        break;
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[0]
 [0]]
Local minimum occurs at: [[0]
 [0]]
Number of steps: 0
runtime:  0.0020024776458740234


### 3) func = 100*(x2 - x1**2)^2 + (1-x1)^2 and x0 = -1.2,1

In [35]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('-1.2;1')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = 100*(x2 - x1**2)**2 + (1-x1)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [44]:
# Start at
x = np.matrix('-1.2;1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = 0.00001
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for gradint descent
while np.linalg.norm(xnew - xold) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
    
    # If gradient is 0 then we stop either way. Or else it will break the code.
    if(np.linalg.norm(fprimeVal) == 0) :
        break;
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[-1.2]
 [ 1. ]]


Iteration:  1
Direction:  [[ 215.6]
 [  88. ]]
Step Length:  0.0006646686267390898
New Value:  [[-1.05669744]
 [ 1.05849084]]




Iteration:  2
Direction:  [[ 28.6789261 ]
 [ 11.62372983]]
Step Length:  0.0006645589342220539
New Value:  [[-1.03763861]
 [ 1.06621549]]




Iteration:  3
Direction:  [[ 8.42438883]
 [ 2.09567743]]
Step Length:  0.0006715724320006056
New Value:  [[-1.03198102]
 [ 1.06762289]]




Iteration:  4
Direction:  [[ 2.97498852]
 [-0.52761315]]
Step Length:  0.0008845814789065256
New Value:  [[-1.0293494 ]
 [ 1.06715618]]




Iteration:  5
Direction:  [[ 0.93112903]
 [-1.51919736]]
Step Length:  0.012367517558442249
New Value:  [[-1.01783365]
 [ 1.04836748]]




Iteration:  6
Direction:  [[-1.00551796]
 [-2.47642887]]
Step Length:  0.0014397679994601954
New Value:  [[-1.01928136]
 [ 1.04480199]]




Iteration:  7
Direction:  [[ 1.64630762]
 [-1.17350086]]
Step Length:  0.0020179946913472056
New Value:  [[-1.01595912]
 [ 

### 4) func = (x1+x2)^4 + x2^2 and x0 = 2,-2
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [30]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = (x1+x2)**4 + x2**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [33]:
def f2(n,x,s):
    x = x + n*s
    return f(x)

# use scipy fmin function to find ideal step size.
alp = fmin(f2,0.1,(xold,direc), full_output = False, disp = False)

Matrix([
[12*(x1 + x2)**2,     12*(x1 + x2)**2],
[12*(x1 + x2)**2, 12*(x1 + x2)**2 + 2]])

In [ ]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('2,-2')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = 0.00001
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for gradint descent
while np.linalg.norm(xnew - xold) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
    
    # If gradient is 0 then we stop either way. Or else it will break the code.
    if(np.linalg.norm(fprimeVal) == 0) :
        break;
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)